
# Multiclass Classification

Building a neural network to solve a multiclass classification exercise using the PyTorch framework.<br>
Classification is one of the basic machine learning exercises. A trained model aims to predict the class of an input unit with high accuracy. <br>
This neural network uses supervised learning, meaning that the input datasets also provide target labels to train the model with. <br>
<br>
This Notebook has been generated automatically using the JupyterLab extension ***MLProvCodeGen***.
<br>

Original Source Code and inspiration from this article https://janakiev.com/blog/pytorch-iris/ <br>
Original author: N. Janakiev https://github.com/njanakiev Twitter: https://twitter.com/njanakiev
        

### Installs
Install required packages before running

In [ ]:
#pip install numpy===1.22.2 pandas===1.3.3 matplotlib===3.5.1 sklearn==0.0 torch===1.8.0 tqdm===4.60.0 ipywidgets===7.6.5 pytorch-model-summary===0.1.2 ipython===7.31.1 gputil===1.4.0 psutil===5.9.0 py-cpuinfo===8.0.0 prov===2.0.0 pydot===1.4.2 --user
#torch currently not supported with python 3.10, downgrading to python 3.9.7 possibly required

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import GPUtil
import psutil
import cpuinfo
import platform
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_curve, auc, confusion_matrix, f1_score, mean_absolute_error, mean_squared_error 
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import tqdm
from tqdm import trange
import datetime
from datetime import date
import time
from pytorch_model_summary import hierarchical_summary
import json
import ipywidgets as widgets
import webbrowser
import IPython
from IPython.display import display, Image
import prov
from prov.model import ProvDocument
from prov.dot import prov_to_dot
import os

plt.style.use('ggplot')

### Provenance Data 

In [ ]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

d1 = ProvDocument()
d1.set_default_namespace('provenanceexample.org')
#set_experiment_info()
d1.add_namespace('experimentinfo', 'experimentinfo.org')
e_experimentinfo = d1.entity('experiment_info', (
    ('experimentinfo:author', 'Tarek Al Mustafa'),
    ('experimentinfo:email', 'tarek.almustafa@uni-jena.de'),
    ('experimentinfo:title', 'Multiclass Classification'),
    ('experimentinfo:creation_date', str(date.today())),
    ('experimentinfo:task_type', 'MulticlassClassification'),
))

a_setexperimentinfo = d1.activity('set_experiment_info()', datetime.datetime.now(), None)
a_setdate = d1.activity('date.today()')

d1.wasGeneratedBy(e_experimentinfo, a_setexperimentinfo, None, {'ex:fct': 'set'})
d1.wasAssociatedWith('experiment_info', 'Tarek Al Mustafa', None, None)
d1.agent('Tarek Al Mustafa') 
d1.wasInformedBy(a_setexperimentinfo, a_setdate)   


#set_hardware_info()
uname = platform.uname()
sysInfo = str(uname.system +' '+ uname.release +' Version: '+ uname.version +' Machine: '+ uname.machine)
    
svmem = psutil.virtual_memory()

GPUs = GPUtil.getGPUs()
gpuList = []
for gpu in GPUs:
    gpu_id = gpu.id
    gpu_name = gpu.name
    gpuList.append((gpu_id , gpu_name))

        
d1.add_namespace('hardwareinfo', 'hardwareinfo.org')
e_hardwareinfo = d1.entity('hardware_info', (
    ('hardwareinfo:Python Version', cpuinfo.get_cpu_info()['python_version']),
    ('hardwareinfo:CPU', cpuinfo.get_cpu_info()['brand_raw']),
    ('hardwareinfo:RAM',  get_size(svmem.total)),
    ('hardwareinfo:Operating System', sysInfo),
    ('hardwareinfo:GPUs', str(gpuList)),
))
#activity
a_sethardwareinfo = d1.activity('set_hardware_info()', datetime.datetime.now(), None)
a_platform_uname = d1.activity('platform.uname()')
a_cpuinfo = d1.activity('cpuinfo.get_cpu_info()')
a_svmemtotal = d1.activity('svmem.total')
a_getsize = d1.activity('get_size(svmem.total)')
a_GPUtilgetGPU = d1.activity('GPUtil.getGPUs()')
d1.wasGeneratedBy(e_hardwareinfo, a_sethardwareinfo, None, {'experimentinfo:fct': 'set'})
d1.wasInformedBy(a_sethardwareinfo, a_platform_uname)
d1.wasInformedBy(a_sethardwareinfo, a_cpuinfo)
d1.wasInformedBy(a_sethardwareinfo, a_svmemtotal)
d1.wasInformedBy(a_svmemtotal, a_getsize)
d1.wasInformedBy(a_sethardwareinfo, a_GPUtilgetGPU)


#set_packages
cpuInfo_version = !pip list | grep -i py-cpuinfo
pytorch_model_summary_version = !pip list | grep -i pytorch-model-summary

d1.add_namespace('packages', 'packages.org')

e_packages = d1.entity('packages', (
    ('packages:numpy', np.__version__),
    ('packages:pandas', pd.__version__),
    ('packages:matplotlib',  matplotlib.__version__),
    ('packages:sklearn', sklearn.__version__),
    ('packages:torch', torch.__version__),
    ('packages:tqdm', tqdm.__version__),
    ('packages:ipywidgets', widgets.__version__),
    ('packages:pytorch-model-summary', pytorch_model_summary_version[0]),
    ('packages:ipython', IPython.__version__),
    ('packages:gputil', GPUtil.__version__),
    ('packages:psutil', psutil.__version__),
    ('packages:py-cpuinfo', cpuInfo_version[0]),
    ('packages:prov', prov.__version__),  
))
a_setpackages = d1.activity('set_packages()', datetime.datetime.now())
a_getVersion = d1.activity('{package_name}.__version__')
a_getVersion_py_cpuinfo = d1.activity('!pip list | grep -i py-cpuinfo')
a_getVersion_pytorch_model_summary = d1.activity('!pip list | grep -i pytorch-model-summary')
d1.wasGeneratedBy(e_packages, a_setpackages, None, {'packages:fct': 'set'})
d1.wasInformedBy(a_setpackages,a_getVersion)
d1.wasInformedBy(a_setpackages,a_getVersion_py_cpuinfo)
d1.wasInformedBy(a_setpackages,a_getVersion_pytorch_model_summary)

### Data Ingestion

In [ ]:
#Data Ingestion
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
output_dim = len(names)
feature_names = iris['feature_names']

#set_data_ingestion
d1.add_namespace('dataingestion', 'dataingestion.org')

e_dataingestion = d1.entity('data_ingestion', (
    ('dataingestion:dataset_id', 'Iris'),
    ('dataingestion:output_dimensions', output_dim),
    ('dataingestion:samples_total',  len(X)),
    ('dataingestion:feature_dimensions', len(feature_names)),  
))
a_setdataingestion = d1.activity('set_data_ingestion()', datetime.datetime.now())
d1.wasGeneratedBy(e_dataingestion, a_setdataingestion, None, {'packages:fct': 'set'})	

### Data Preparation

In [ ]:
#Data Preperation
# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#set_data_preparation
d1.add_namespace('datapreparation', 'datapreparation.org')

e_datapreparation = d1.entity('data_preparation', (
    ('datapreparation:number_operations', 1),
    ('datapreparation:transformation', 'sklearn.preprocessing.StandardScaler'),
    ('datapreparation:transformation_method', 'Standardscaler.fit_transform'), 
))
a_setdatapreparation = d1.activity('set_data_preparation()', datetime.datetime.now())
d1.wasGeneratedBy(e_datapreparation, a_setdatapreparation, None, {'packages:fct': 'set'})
d1.used(a_setdatapreparation, e_dataingestion)

### Data Segregation

In [ ]:
# Data Segregation
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)
	
#set_data_segregation
d1.add_namespace('datasegregation', 'datasegregation.org')

e_datasegregation = d1.entity('data_segregation', (
    ('datasegregation:segregation_method', 'sklearn.model_selection.train_test_split'),
    ('datasegregation:test_size', 0.2),
    ('datasegregation:train_size', 1-0.2), 
    ('datasegregation:random_state', 2), 
))
a_setdatasegregation = d1.activity('set_data_segregation()', datetime.datetime.now())
d1.wasGeneratedBy(e_datasegregation, a_setdatasegregation, None, {'packages:fct': 'set'})
d1.used(a_setdatasegregation, e_datapreparation)

### Data Visualization

In [ ]:
#Data Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
for target, target_name in enumerate(names):
    X_plot = X[y == target]
    ax1.plot(X_plot[:, 0], X_plot[:, 1], 
             linestyle='none', 
             marker='o', 
             label=target_name)
ax1.set_xlabel(feature_names[0])
ax1.set_ylabel(feature_names[1])
ax1.axis('equal')
ax1.legend();

for target, target_name in enumerate(names):
    X_plot = X[y == target]
    ax2.plot(X_plot[:, 2], X_plot[:, 3], 
             linestyle='none', 
             marker='o', 
             label=target_name)
ax2.set_xlabel(feature_names[2])
ax2.set_ylabel(feature_names[3])
ax2.axis('equal')
ax2.legend();


### Model

In [ ]:
#Use GPU?
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

neuron_number = 50

#Configure Neural Network Models
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, output_dim)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

model     = Model(X_train.shape[1])
lr=0 
optimizer = torch.optim.Adam(model.parameters(), lr=lr) loss_fn   = nn.CrossEntropyLoss()

try:
    lr
except NameError:
    default = 1
    lr = None 
else:
    default = 0

#set_model_parameters
d1.add_namespace('modelparameters', 'modelparameters.org')

e_modelparameters = d1.entity('model_parameters', (
    ('modelparameters:gpu_enable', 1),
    ('modelparameters:modelParameters', str(model)),
    ('modelparameters:neuron_number', neuron_number), 
    ('modelparameters:loss_function', 'nn.CrossEntropyLoss()'), 
    ('modelparameters:optimizer', 'torch.optim.Adam('), 
    ('modelparameters:optimizer_default_learning_rate', default), 
    ('modelparameters:optimizer_learning_rate', str(lr)), 
    ('modelparameters:activation_function', 'F.softmax(self.layer3(x), dim=1)'), 
))
a_setmodelparameters = d1.activity('set_model_parameters()', datetime.datetime.now())
d1.wasGeneratedBy(e_modelparameters, a_setmodelparameters, None, {'packages:fct': 'set'})
d1.used(a_setmodelparameters, e_dataingestion)

model

### Training

In [ ]:
#Model Training
EPOCHS  = 100
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
X_test  = Variable(torch.from_numpy(X_test)).float()
y_test  = Variable(torch.from_numpy(y_test)).long()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()

#set_training
d1.add_namespace('training', 'training.org')

e_training = d1.entity('training', (
    ('training:epochs', EPOCHS),
    ('training:numberOfParameters', hierarchical_summary(model, print_summary = False)[1]),
))
a_settraining = d1.activity('set_training()', datetime.datetime.now())
a_hierarchichalsummary = d1.activity('hierarchical_summary(model, print_summary = False)[1]')
d1.wasGeneratedBy(e_training, a_settraining, None, {'packages:fct': 'set'})
d1.wasInformedBy(a_settraining, a_hierarchichalsummary)
d1.used(a_settraining, e_modelparameters)
d1.used(a_settraining, e_datasegregation)

### Evaluation

In [ ]:
#Plot Accuracy and Loss from Training
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 6), sharex=True)

ax1.plot(accuracy_list)
ax1.set_ylabel("validation accuracy")
ax2.plot(loss_list)
ax2.set_ylabel("validation loss")
ax2.set_xlabel("epochs");

### Confusion Matrix

In [ ]:
confusionMatrix = confusion_matrix(y_test, torch.argmax(y_pred, dim=1))
confusionMatrix

### F1 Score

In [ ]:
F1 = f1_score(y_test, torch.argmax(y_pred, dim=1), average=None)
F1

### Mean Absolute Error & Mean Squared Error

In [ ]:
MAE = mean_absolute_error(y_test, torch.argmax(y_pred, dim=1), multioutput='uniform_average')
MSE = mean_squared_error(y_test, torch.argmax(y_pred, dim=1), multioutput='uniform_average')
print(MAE)
print(MSE)

### ROC

In [ ]:
#ROC Curve 
plt.figure(figsize=(10, 10))
plt.plot([0, 1], [0, 1], 'k--')

# One hot encoding
enc = OneHotEncoder()
Y_onehot = enc.fit_transform(y_test[:, np.newaxis]).toarray()

with torch.no_grad():
    y_pred = model(X_test).numpy()
    fpr, tpr, threshold = roc_curve(Y_onehot.ravel(), y_pred.ravel())
    
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc(fpr, tpr)))
AUC = '{:.5f}'.format(auc(fpr, tpr))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend();

#set_evaluation
d1.add_namespace('evaluation', 'evaluation.org')

e_evaluation = d1.entity('evaluation', (
    ('evaluation:Accuracy', accuracy_list[(len(accuracy_list)-1)]),
    ('evaluation:Loss', loss_list[(len(loss_list)-1)]),
    ('evaluation:Confusion Matrix', str(confusionMatrix)),
    ('evaluation:AUC', float(AUC)),
    ('evaluation:F1 Score', str(F1)),
    ('evaluation:Mean Absolute Error', MAE),
    ('evaluation:Mean Squared Error', MSE),
))
a_settraining = d1.activity('set_evaluation()', datetime.datetime.now())
d1.wasGeneratedBy(e_evaluation, a_settraining, None, {'packages:fct': 'set'})
d1.used(a_settraining, e_training)

### Generate Provenance Data

In [ ]:
#add visualization to PATH
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

#get time for filenames
timestring = time.strftime('%Y%m%d-%H%M%S')
timestring
ProvenanceNameImage = ('Provenance_MulticlassClassification_' + timestring + '.png')

dot = prov_to_dot(d1)
dot.write_png('../GeneratedProvenanceData/'+ProvenanceNameImage)

provenanceImage_open = widgets.Button(description = 'Open Image File')
display(provenanceImage_open)

def on_button_clicked(b):
    provenanceImage_open.on_click = webbrowser.open('http://localhost:8888/lab/tree/extension/GeneratedProvenanceData/'+ProvenanceNameImage)

provenanceImage_open.on_click(on_button_clicked)
Image('../GeneratedProvenanceData/'+ProvenanceNameImage)

### Write Provenance Data

In [ ]:
ProvenanceName = ('Provenance_MulticlassClassification_' + timestring + '.json')

with open('../GeneratedProvenanceData/'+ProvenanceName, 'w') as prov_file:
    prov_file.write(d1.serialize(indent=2))

### Show Provenance Data

In [ ]:
provenance_open = widgets.Button(description = 'Open Provenance Data File')
display(provenance_open)

def on_button_clicked(b):
    #provenance_open.on_click = webbrowser.open('http://localhost:8888/lab/tree/GeneratedProvenanceData/'+ProvenanceName)
	provenance_open.on_click = webbrowser.open('http://localhost:8888/lab/tree/extension/GeneratedProvenanceData/'+ProvenanceName)

provenance_open.on_click(on_button_clicked)